# deployment steps:
0) 
1) make sure that Dockerfile, requirements.txt and main.py is in repo
2) export vars to ~/.zshrc
!!!full paths to the credentials.json and token.json this is for local run
change here GOOGLE_CLOUD_PROJECT, GOOGLE_OAUTH_CLIENT_FILE, GOOGLE_OAUTH_TOKEN_FILE
```
export GOOGLE_GENAI_USE_VERTEXAI=True
export GOOGLE_CLOUD_PROJECT=personalplanner-476218       
export GOOGLE_CLOUD_LOCATION=us-central1
export GOOGLE_OAUTH_CLIENT_FILE="/Users/gman/Documents/hackathon/personalPlanner/.creds/credentials.json"  
export GOOGLE_OAUTH_TOKEN_FILE="/Users/gman/Documents/hackathon/personalPlanner/.creds/token.json"
export MODEL=gemini-2.5-flash
```
3) ```source ~/.zshrc```
NOTE to run the current verison of setup run with python main.py. the above exports are for local run only, not cloudrun!!!

4) deploy containers ( change project name in 2 places here)

```
gcloud run deploy project-planner-service \
  --source . \
  --region us-central1 \
  --project personalplanner-476218 \
  --allow-unauthenticated \
  --set-env-vars=GOOGLE_GENAI_USE_VERTEXAI=True,GOOGLE_CLOUD_PROJECT=personalplanner-476218,GOOGLE_CLOUD_LOCATION=us-central1,MODEL=gemini-2.5-flash,GOOGLE_OAUTH_CLIENT_FILE=/app/.creds/credentials.json,GOOGLE_OAUTH_TOKEN_FILE=/app/.creds/token.json


```

5) run this ( Change project name )
```
gcloud run services describe project-planner-service \
  --region=us-central1 \
  --project=personalplanner-476218 \
  --format='value(spec.template.spec.serviceAccountName)'


```


it returns something like xxxxxxx-compute@developer.gserviceaccount.com
then run what's below with that thingy as your SA_EMAIL
```
SA_EMAIL="98380938461-compute@developer.gserviceaccount.com"      

gcloud projects add-iam-policy-binding personalplanner-476218 \
  --member="serviceAccount:${SA_EMAIL}" \
  --role="roles/aiplatform.user"

```



# Paste your service URL here 

In [28]:
import requests
import json
# Service [project-planner-service] revision [project-planner-service-00002-fhq] has been deployed and is serving 100 percent of traffic.
# Service URL: https://project-planner-service-98380938461.us-central1.run.app
SERVICE_URL = "https://project-planner-service-98380938461.us-central1.run.app"

# Helpers

In [29]:
APP_NAME = "orchestrator"  # from /list-apps
USER_ID = "grant"          # anything you like
SESSION_ID = "sess1"       # anything you like
def pretty_print(obj):
    print(json.dumps(obj, indent=2, ensure_ascii=False))


# 2) List available apps
def list_apps():
    url = f"{SERVICE_URL}/list-apps"
    resp = requests.get(url)
    resp.raise_for_status()
    apps = resp.json()
    print("Available apps:")
    pretty_print(apps)
    return apps


# 3) Create or update a session

def create_session(initial_state=None):
    if initial_state is None:
        initial_state = {}

    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.post(url, json=initial_state)

    print("Status:", resp.status_code)
    print("Headers:", resp.headers)
    print("Body:\n", resp.text)

    # Only raise if it's 4xx/5xx *after* we print
    resp.raise_for_status()

    try:
        session = resp.json()
        print("\nParsed session JSON:")
        pretty_print(session)
        return session
    except Exception as e:
        print("\nJSON parse error:", e)
        return None



# 4) Get session (optional, for debugging)
def get_session():
    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.get(url)
    resp.raise_for_status()
    session = resp.json()
    print("Current session:")
    pretty_print(session)
    return session


# 5) Run the orchestrator with a user message
def run_orchestrator(message_text: str):
    url = f"{SERVICE_URL}/run"
    payload = {
        "app_name": APP_NAME,
        "user_id": USER_ID,
        "session_id": SESSION_ID,
        "new_message": {
            "role": "user",
            "parts": [
                {"text": message_text}
            ]
        }
    }
    resp = requests.post(url, json=payload)
    print("Status:", resp.status_code)
    print("Body:\n", resp.text)
    resp.raise_for_status()
    events = resp.json()
    print("Events from orchestrator:")
    pretty_print(events)
    return events


# 6) Convenience helper: full flow in one call
def send_to_orchestrator(message_text: str, init_if_needed: bool = True):
    """
    - Optionally creates/updates a session first
    - Sends a message
    - Returns events
    """
    if init_if_needed:
        # you can attach any initial session state here if you want
        create_session(initial_state={})

    return run_orchestrator(message_text)


# 7) Delete session (if you want to reset state)
def delete_session():
    url = f"{SERVICE_URL}/apps/{APP_NAME}/users/{USER_ID}/sessions/{SESSION_ID}"
    resp = requests.delete(url)
    if resp.status_code not in (200, 204):
        print("Delete returned:", resp.status_code, resp.text)
    else:
        print("Session deleted.")



In [30]:
apps = list_apps()

Available apps:
[
  "TESTING_apollo_service",
  "calendar_service",
  "gmail_service",
  "google_docs_service",
  "google_drive_service",
  "google_search_service",
  "google_sheets_service",
  "jobs_service",
  "orchestrator",
  "resume_customization",
  "ui",
  "utils"
]


In [35]:
create_session(initial_state={"note": "session created from Jupyter"})


Status: 409
Headers: {'content-type': 'application/json', 'x-cloud-trace-context': '4a5a8736d54b84e07660e7f6d0b34044;o=1', 'date': 'Sat, 08 Nov 2025 09:45:37 GMT', 'server': 'Google Frontend', 'Content-Length': '42', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}
Body:
 {"detail":"Session already exists: sess1"}


HTTPError: 409 Client Error: Conflict for url: https://project-planner-service-98380938461.us-central1.run.app/apps/orchestrator/users/grant/sessions/sess1

In [32]:
events = run_orchestrator(
    "Help me plan my tasks for tomorrow. I have meetings 10–12 and want deep work in the afternoon."
)


Status: 200
Body:
 [{"modelVersion":"gemini-2.5-flash","content":{"parts":[{"functionCall":{"id":"adk-faa72b7f-b54e-452d-8473-29271914dfab","args":{"agent_name":"google_calendar_agent"},"name":"transfer_to_agent"},"thoughtSignature":"CvECAePx_16ygQOJsLmE8PmV6pp5OUMrHAnCDnQ7GFwTJ-qCItPDP5nrCGUGz_FZPhGxbzbgC43mq8BhP6wj0r97mLiL1fOtYgrPQJ3nK1qaLVtn9JDjysU9MATy-dVOhVPGBUTRodjXUu2wvIQjwQrz7ve8zGQllI1ZclCyAd4jVl2Pj0GvxFgCFtGEn3mbt5u4PPsbj-7xAQbsYXSfzCy-DmA6jnLJ77cAQcmrYTt18ziCVee8yoVJExS3Eaf5BJpVPbuzQpGNl1zZmIe2xyVo9nDHUKIDGqYNAojckCenGO61rVgLVNWf-t0cfJEMliIZEIp5rnjnQsF34wdPLuBTo4IS1bfm01CaaadJ6F5iUSAG5ocZKqyEYdFRzfVl7eSeQsp1vY_CFC7nOy7uL2FFCIHvDenV01fxUctuhXethDfQSj_YfV-9idNj53cdjzHOPzaUfuhT-ha_FDBf2FV3r0yzcz9Z-MTA5CdlxWlUkRa6"}],"role":"model"},"finishReason":"STOP","usageMetadata":{"candidatesTokenCount":13,"candidatesTokensDetails":[{"modality":"TEXT","tokenCount":13}],"promptTokenCount":1983,"promptTokensDetails":[{"modality":"TEXT","tokenCount":1983}],"thoughtsTokenCount":63,"totalToken

In [33]:
events = run_orchestrator(
    "hey actually can you schedule a meeting in my calendar at 2 - 3 om today with title Meet with Steven?."
)

Status: 200
Body:
 [{"modelVersion":"gemini-2.5-flash","content":{"parts":[{"functionCall":{"id":"adk-4c0d4f62-1597-47d8-a005-bc4d5b3e384d","args":{},"name":"make_time_context"},"thoughtSignature":"Cv8DAePx_15gYVIL2vXL-iCLu-wpyzoxynRQePCVYavv91kPX3XdUkmp18uPZM6PYKu5tmpEb_p9XbRlVuS_gH_5iDqAeOYNEbVApb_lZyoVSQMbrdki-qWHE6_HSr5gqV2W0UNE5UDoR4USDgeYCuvC4YkIjDA6s4iKiLJz8txOMndR5LQPEow4oto8cqib8AUrEgFi5cfyjkADxmVz9HyQv4a00l0iOSq5UFHmEEoIT9MrFwp4pTwatPw78MGI-H_5qwryxRLltcpOgHGY4EjCb7iHS7Q1VB_1lh56D8W8nVpw1pTxSjKcPmZCkFjGmPBmJmrcdrNm2Ty5jvo70ITnWjP_Iz5D1oj9mQLLFWyPZZvMlWcNI5tgzrObMwzxHrLPd0Xk31LMKoJUVqoiU69v718hvNpGwLz0NQR17gmVJoYr59ytbQFJ5URLnLqhhgVfUYziNke3foltV2bhR1BoV5jSyVq-tmVojQlXrbM-gu5WIeKWjgo69Lz0HVH8RA5a2yBarVPYEAsRfw_S6W3Ibw2DelplKxzJWU8_eyhR3U99PosaXU0fx0-lGkomkgHoiTm4Br5YPVsJQbvAdgvY74xmIPzesNImoLOl8lSxw0GM1_wsGRswg2_Dt13eKlgDROYoKmJ76BzEOrxBMB8ICoji9-VGH8zDnAqSi5arBA=="}],"role":"model"},"finishReason":"STOP","usageMetadata":{"candidatesTokenCount":5,"candidatesTokensDetails":[{"m

In [34]:
events = run_orchestrator(
    "hey actually can you schedule a meeting in my calendar at 3 - 4 om today with title STEEVOO."
)

Status: 200
Body:
 [{"modelVersion":"gemini-2.5-flash","content":{"parts":[{"functionCall":{"id":"adk-ad4d4b50-1dfe-45a8-8580-5d6b6be0ac36","args":{"end_datetime":"2025-11-08T16:00:00","start_datetime":"2025-11-08T15:00:00","summary":"STEEVOO"},"name":"create_event"},"thoughtSignature":"CvUCAePx_15W8xd5KR8nXfcb3ZX8GJugslZ_8ux4mEV07nOQm6X14e75pvpTYkIqM2XQxA_-AFULzn-1U0e_DktXXJKFF37sBoSujAV276Fq1Nep6oe8sBa9_nurvNhJBC8dbXnAtv8_2X299JNfUDUJ3hQXioa_uSNQ_9R9INV-A2dug3luU0kDTODP9QRqLW9tNmKaw9BcaZ6xK4KQFATV_h_HYjMByg7n_bynWavliI9Wk2LK6a3NpARcUZCKpPfx1yhlrKYEYwxgxYEiECRGGttwXQaOdecOiJeS14I9-84Aw_H0xJGcEduiFgZBlB56YUzUf-Dvdq1S09sW73NzIl6mSkJvTzws5-AxogDEWGM6-Wg309WMisv50BYPPeVT3u4C8juiCFywXs9jcOC092YaRXAAFFNtPSFDKf334GUqCz0HCTaZ_rtVR2AQ-g2bKJ-KgFeTTaJRiNWIA33uavMfLYEXLpDhXIJ7pHb94BOUrIT6pw=="}],"role":"model"},"finishReason":"STOP","usageMetadata":{"cacheTokensDetails":[{"modality":"TEXT","tokenCount":3010}],"cachedContentTokenCount":3010,"candidatesTokenCount":52,"candidatesTokensDetails":[{"mo